In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms

import glob
import os

import PIL.Image
from torch.autograd import Variable

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cuda'

In [3]:
model = torch.load('I:\\ML\\Projs\\videoclass\\models\\resnet50accuracy78.pth')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [4]:


imsize = 256
loader = transforms.Compose([transforms.Resize(255),
                             transforms.CenterCrop(224),
                             transforms.ToTensor(),
                             transforms.Normalize([0.485, 0.456, 0.406],
                                                  [0.229, 0.224, 0.225])])

In [5]:
def image_loader(image_name):
    """load image, returns cuda tensor"""
    fp = open(image_name,"rb")
    image = PIL.Image.open(fp)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

In [6]:
def image_predict(image_path):
    image = image_loader(image_path)
    out = model(image)
    index = out.data.cpu().numpy().argmax()
    return index

TypeError: can only concatenate str (not "int") to str

In [7]:
import shutil


a,b = 0,0
path = 'I:\\ML\\Projs\\videoclass\\prediction data'
files = os.listdir(path)
desti = 'I:\\ML\\Projs\\videoclass\\stitch'

for f in files:
    temp = os.path.join(path, f)
    if (image_predict(temp) == 0):
        shutil.move(temp ,os.path.join(desti, f))
        a = a + 1
    else:
        b = b + 1        
        

print("Number of Zeros: {} Number of Ones: {} ".format(a, b) )

Number of Zeros: 10127 Number of Ones: 19873 


1
